# Validación y Calibración del Sensor BME680 vs Estación Meteorológica UNAM

Este notebook documenta el proceso de validación y calibración de un sensor **BME680** (prototipo IoT) comparado contra una estación meteorológica de referencia (**UNAM**).

> **Objetivo:** Demostrar la mejora en precisión tras aplicar una corrección de sesgo (Offset) basada en datos experimentales.

## 0. Vista Previa de Datos (Estática)
Muestra de las primeras 10 filas del dataset `comparacion_detallada.csv` utilizado en este análisis:

| Time_Key | Timestamp | Temp_UNAM | Temp_BME | Temp_Calibrated | Diff_Temp | Diff_Calibrated | Abs_Diff_Temp | Pct_Error_Temp | Global_MAPE | Global_Pearson | Global_RMSE | Global_R2 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 2026-02-11 09:25:00 | 2026-02-11 09:24:43 | 17.68 | 16.7 | 16.87 | -0.98 | -0.81 | 0.98 | 5.543 | 24.9858 | 0.847 | 3.4498 | 0.5296 |
| 2026-02-11 08:39:00 | 2026-02-11 08:39:16 | 15.13 | 15.5 | 15.67 | 0.37 | 0.54 | 0.37 | 2.4455 | 24.9858 | 0.847 | 3.4498 | 0.5296 |
| 2026-02-11 08:24:00 | 2026-02-11 08:24:14 | 15.46 | 13.3 | 13.47 | -2.16 | -1.99 | 2.16 | 13.9715 | 24.9858 | 0.847 | 3.4498 | 0.5296 |
| 2026-02-11 08:09:00 | 2026-02-11 08:09:03 | 13.74 | 11.6 | 11.77 | -2.14 | -1.97 | 2.14 | 15.575 | 24.9858 | 0.847 | 3.4498 | 0.5296 |
| 2026-02-11 07:24:00 | 2026-02-11 07:23:40 | 10.6 | 8.3 | 8.47 | -2.3 | -2.13 | 2.3 | 21.6981 | 24.9858 | 0.847 | 3.4498 | 0.5296 |
| 2026-02-11 06:23:00 | 2026-02-11 06:23:24 | 10.18 | 8.1 | 8.27 | -2.08 | -1.91 | 2.08 | 20.4322 | 24.9858 | 0.847 | 3.4498 | 0.5296 |
| 2026-02-11 06:08:00 | 2026-02-11 06:08:24 | 10.47 | 8.3 | 8.47 | -2.17 | -2.0 | 2.17 | 20.7259 | 24.9858 | 0.847 | 3.4498 | 0.5296 |
| 2026-02-11 05:53:00 | 2026-02-11 05:53:24 | 10.37 | 8.3 | 8.47 | -2.07 | -1.9 | 2.07 | 19.9614 | 24.9858 | 0.847 | 3.4498 | 0.5296 |
| 2026-02-11 04:38:00 | 2026-02-11 04:37:54 | 11.18 | 9.2 | 9.37 | -1.98 | -1.81 | 1.98 | 17.7102 | 24.9858 | 0.847 | 3.4498 | 0.5296 |
| 2026-02-11 04:23:00 | 2026-02-11 04:22:54 | 11.43 | 8.9 | 9.07 | -2.53 | -2.36 | 2.53 | 22.1347 | 24.9858 | 0.847 | 3.4498 | 0.5296 |

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Configuración de estilo
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]

## 1. Carga de Datos
Cargamos el dataset alineado temporalmente, que incluye mediciones crudas, de referencia y calibradas.

In [ ]:
# Cargar dataset
df = pd.read_csv('comparacion_detallada.csv')

# Convertir fecha
df['Time_Key'] = pd.to_datetime(df['Time_Key'])

# Mostrar estructura
display(df[['Time_Key', 'Temp_UNAM', 'Temp_BME', 'Temp_Calibrated']].head())

## 2. Evaluación de Mejora (Antes vs Después)
Comparamos las métricas de error usando los datos crudos vs los datos con el offset aplicado.

In [ ]:
# Cálculo de métricas para Datos Calibrados
cal_rmse = np.sqrt(((df['Temp_Calibrated'] - df['Temp_UNAM']) ** 2).mean())
cal_mape = (np.abs((df['Temp_Calibrated'] - df['Temp_UNAM']) / df['Temp_UNAM'])).mean() * 100

# Métricas originales (ya calculadas en CSV o recalculadas aquí)
orig_rmse = df['Global_RMSE'].iloc[0]
orig_mape = df['Global_MAPE'].iloc[0]
offset = df['Diff_Temp'].mean()

diff_rmse = orig_rmse - cal_rmse

print(f"=== IMPACTO DE CALIBRACIÓN (Offset: {offset:.2f} °C) ===")
print(f"{'-':<20} | {'Original':<10} | {'Calibrado':<10} | {'Mejora':<10}")
print(f"RMSE (Error Abs)     | {orig_rmse:.2f} °C     | {cal_rmse:.2f} °C     | {diff_rmse:.2f} °C")
print(f"MAPE (Error %)       | {orig_mape:.2f} %      | {cal_mape:.2f} %      | -\n")

print("Nota: El Offset corrige el error sistemático (Bias), reduciendo el RMSE.")

## 3. Visualización de Resultados

### 3.1 Serie de Tiempo: Corrección del Sesgo
Gráfica que muestra cómo la señal calibrada (Verde) se ajusta mejor a la referencia (Negra) que la señal original (Roja).

In [ ]:
plt.figure(figsize=(14, 7))

# Referencia
sns.lineplot(data=df, x='Time_Key', y='Temp_UNAM', label='Referencia (UNAM)', color='black', linewidth=2, alpha=0.8)

# Sensor Original
sns.lineplot(data=df, x='Time_Key', y='Temp_BME', label='Sensor Crudo (BME680)', color='red', linestyle='--', alpha=0.5)

# Sensor Calibrado
sns.lineplot(data=df, x='Time_Key', y='Temp_Calibrated', label='Sensor Calibrado', color='green', linewidth=2, alpha=0.9)

plt.title('Efecto de la Calibración: Alineación de Temperatura')
plt.ylabel('Temperatura (°C)')
plt.xlabel('Tiempo')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 3.2 Histograma de Error Residual
Comparamos la distribución del error antes y después. Idealmente, el error calibrado debe estar centrado en 0.

In [ ]:
plt.figure(figsize=(12, 6))

sns.histplot(df['Diff_Temp'], color='red', label='Error Original', kde=True, alpha=0.3, binwidth=0.2)
sns.histplot(df['Diff_Calibrated'], color='green', label='Error Calibrado', kde=True, alpha=0.5, binwidth=0.2)

plt.axvline(x=0, color='black', linestyle='--', linewidth=1, label='Cero Error (Ideal)')

plt.title('Distribución del Error: Antes vs Después')
plt.xlabel('Error (°C) [Sensor - Referencia]')
plt.legend()
plt.show()

### 3.3 Correlación Final
Confirmamos que la linealidad se mantiene alta.

In [ ]:
plt.figure(figsize=(8, 8))
sns.scatterplot(data=df, x='Temp_UNAM', y='Temp_Calibrated', alpha=0.6, color='green')

# Línea de identidad
lims = [5, 30]
plt.plot(lims, lims, '--k', label='Ideal (y=x)')

plt.title(f"Correlación Final (Calibrada) | Pearson r: {df['Global_Pearson'].iloc[0]:.4f}")
plt.xlabel('Referencia UNAM (°C)')
plt.ylabel('Sensor Calibrado (°C)')
plt.legend()
plt.grid(True)
plt.axis('equal')
plt.show()

## 4. Conclusión Final
La aplicación del offset calculado ha centrado la distribución del error en cero, mejorando significativamente la exactitud absoluta (RMSE) del sistema sin alterar su capacidad de respuesta a cambios (Correlación).